# Cytoscape example
> An example of graph visualization with Cytoscape.

In [1]:
import deetly
from cyjupyter import Cytoscape
import pandas as pd

In [2]:
# Create datapackage
description = """
# Example graph with cytoscape
"""

package = {
    "name":"Example graph with cytoscape", 
    "description":description, 
    "author":"Paul Bencze", 
    "theme":"example",
    "license": "MIT",
    "keyword": ["example, graph, cytoscape"]
}

dp = deetly.package(package)

Missing mandatory props(s): ['title']
Missing recommended props(s): ['contactPoint', 'distribution', 'publisher', 'spatial', 'temporal']
Property not in DCAT schema: ['name', 'author', 'license']
Invalid type: ['String:name', 'String:description', 'String:author', 'String or list of strings:theme', 'String:license', 'String or list of strings:keyword', 'ISO date or datetime:issued', 'ISO date or datetime:modified']


In [3]:
df_edges = pd.read_csv("https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-all-edges.csv")
df_nodes = pd.read_csv("https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-all-nodes.csv")

In [4]:
df_nodes.head(1)

,Id,Label
0,Addam-Marbrand,Addam Marbrand


In [5]:
df_edges.head(1)

,Source,Target,Type,id,weight
0,Addam-Marbrand,Brynden-Tully,Undirected,0,3


In [6]:
from graphviz import Digraph
dot = Digraph(comment='VIP graph')

for node in df_nodes:
    dot.node(node)
    
for i in range(len(df_edges)):
    edge = df_edges.iloc[i]
    dot.edge(edge['Source'], edge['Target'])

In [7]:
dot

KeyboardInterrupt: 

In [7]:
def getNode(row):
    return {'data':{'id':row['Id']}}
    #return {'data':{'id':row['Id'],'label':row['Label']}}
    
df_nodes['node'] = df_nodes.apply(getNode, axis=1)

In [8]:
nodes = df_nodes['node'].to_list()
nodes[0]

{'data': {'id': 'Addam-Marbrand'}}

In [9]:
df_edges.head(1)

,Source,Target,Type,id,weight
0,Addam-Marbrand,Brynden-Tully,Undirected,0,3


In [10]:
def getEdge(row):
    return {'data':{'source':row['Source'],'target':row['Target']}}
    #return {'data':{'id':row['id'],'source':row['Source'],'target':row['Target'],'type':row['Type'],'weight':row['weight']}}
    
df_edges['edge'] = df_edges.apply(getEdge, axis=1)

In [11]:
edges = df_edges['edge'].to_list()

In [12]:
network = {
    'elements': {
        'nodes': nodes[0:100],
        'edges': edges[0:200]
    }
}

In [13]:
network['elements']['nodes'][1]

{'data': {'id': 'Aegon-Frey-(son-of-Stevron)'}}

In [14]:
fig = Cytoscape(data=network, layout_name='cose')
fig

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'Addam-Marbrand'}}, {'data': {'id': 'Aegon-Frey-(son-of…

In [17]:
import networkx as nx
from networkx.readwrite import cytoscape_data
from networkx.algorithms import betweenness_centrality, edge_betweenness_centrality

In [18]:
network1 = nx.scale_free_graph(100)
cyjs1 = cytoscape_data(network1)
Cytoscape(data=cyjs1)

Cytoscape(data={'data': [], 'directed': True, 'multigraph': True, 'elements': {'nodes': [{'data': {'id': '0', …

In [19]:
G = nx.waxman_graph(150)

# Calculate betweenness centrarity of the graph (for both nodes and edges)
bc = betweenness_centrality(G)
ebc = edge_betweenness_centrality(G)
nx.set_node_attributes(G, bc, 'betweenness')
nx.set_edge_attributes(G, ebc, 'betweenness')

# Pick min and max for each data
bc_min = min(bc.values())
bc_max = max(bc.values())
ebc_min = min(ebc.values())
ebc_max = max(ebc.values())

# Create visual mappings

# For nodes
mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', 5, 50)'
op_mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', 0.3, 1)'
color_mapper = 'mapData(betweenness,' + str(bc_min) + ',' + str(bc_max) + ', #EFEFEF, #00897B)'

# For edges
edge_mapper = 'mapData(betweenness,' + str(ebc_min) + ',' + str(ebc_max) + ', 1, 10)'
edge_op_mapper = 'mapData(betweenness,' + str(ebc_min) + ',' + str(ebc_max) + ', 0.3, 1)'

# Build a style
custom_visual_style = [
    {
        'selector': 'node',
        'style': {
            'background-color': color_mapper,
            'width': mapper,
            'height': mapper,
            'background-opacity': op_mapper,
            'label': "data(id)"
        }
    },
    {
        'selector': 'edge',
        'style': {
            'width': edge_mapper,
            'opacity': edge_op_mapper,
            'line-color': '#333333'        
        }
    }
]

# Convert to Cytoscape.js JSON
cyjs2 = cytoscape_data(G)

# Visualize it
Cytoscape(data=cyjs2,  layout_name='circle', visual_style=custom_visual_style, background='#EEEEEE')

Cytoscape(background='#EEEEEE', data={'data': [], 'directed': False, 'multigraph': False, 'elements': {'nodes'…

In [20]:
import requests
url = 'https://raw.githubusercontent.com/plotly/dash-cytoscape/master/demos/data/sample_network.txt'
data = requests.get(url).text.split('\n')

In [21]:
nodes = set()
cy_edges, cy_nodes = [], []
edges = data[:200]
colors = ['red', 'blue', 'green', 'yellow', 'pink']

for edge in edges:
    source, target = edge.split(" ")
    color = colors[len(cy_nodes) % 5]

    if source not in nodes:  # Add the source node
        nodes.add(source)
        cy_nodes.append({"data": {"id": source}, "classes": color})

    if target not in nodes:  # Add the target node
        nodes.add(target)
        cy_nodes.append({"data": {"id": target}, "classes": color})

    cy_edges.append({  # Add the Edge Node
        'data': {'source': source, 'target': target},
        'classes': color
    })

In [22]:
network = {
    'elements': {
        'nodes': cy_nodes,
        'edges': cy_edges
    }
}

In [189]:
fig = Cytoscape(data=network)
fig

Cytoscape(data={'elements': {'nodes': [{'data': {'id': '108082478497335384404'}, 'classes': 'red'}, {'data': {…

In [23]:
fig.get_view_spec()

{'version_major': 2,
 'version_minor': 0,
 'model_id': '7d0d2cf47ec346938d63c3e7a556faf5'}

In [24]:
fig.get_state()

{'_dom_classes': (),
 '_model_module': 'cytoscape-jupyter-widget',
 '_model_module_version': '^0.1.0',
 '_model_name': 'CytoscapeModel',
 '_view_count': None,
 '_view_module': 'cytoscape-jupyter-widget',
 '_view_module_version': '^0.1.0',
 '_view_name': 'CytoscapeView',
 'background': '#FFFFFF',
 'data': {'elements': {'nodes': [{'data': {'id': 'Addam-Marbrand'}},
    {'data': {'id': 'Aegon-Frey-(son-of-Stevron)'}},
    {'data': {'id': 'Aegon-I-Targaryen'}},
    {'data': {'id': 'Aegon-Targaryen-(son-of-Rhaegar)'}},
    {'data': {'id': 'Aegon-V-Targaryen'}},
    {'data': {'id': 'Aemon-Targaryen-(Dragonknight)'}},
    {'data': {'id': 'Aemon-Targaryen-(Maester-Aemon)'}},
    {'data': {'id': 'Aenys-Frey'}},
    {'data': {'id': 'Aeron-Greyjoy'}},
    {'data': {'id': 'Aerys-I-Targaryen'}},
    {'data': {'id': 'Aerys-II-Targaryen'}},
    {'data': {'id': 'Aggar'}},
    {'data': {'id': 'Aggo'}},
    {'data': {'id': 'Alayaya'}},
    {'data': {'id': 'Albett'}},
    {'data': {'id': 'Alebelly'}},
  

In [198]:
## Publish

In [199]:
dp.publish()

Public datapackage: https://storage.googleapis.com/deetly/examples/0be9a787c00f0569d669fb2492424fbc/datapackage.json 

Published at: https://public.deetly.com/examples/0be9a787c00f0569d669fb2492424fbc 

